In [1]:
# Reworking the raw data and training a NN on that, in case I have thrown out too many features.
# Import our dependencies

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import tensorflow as tf


In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head(5)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN'])
application_df.head(5)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:

# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [6]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [7]:
# since there isn't much variation in the STATUS and SPECIAL_CONSIDERATIONS
# columns, they aren't important for our purposes
application_df = application_df.drop(columns=['STATUS', 'SPECIAL_CONSIDERATIONS'])
application_df.head(2)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1


In [8]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
name_counts
     

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [9]:
name_counts[name_counts > 5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [10]:
# Determine which values to replace if counts are less than or equal to 5.
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for name in names_to_replace:
  application_df['NAME'] = application_df['NAME'].replace(name, 'Other')
    
# Check to make sure binning was successful
application_df['NAME'].value_counts()
     

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [11]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# Look at CLASSIFICATION value counts for binning
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# Determine which values to replace if counts are less than 1000
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:

# Generate our categorical variable lists
categvar = list(application_df.dtypes[application_df.dtypes == 'object'].index)
categvar
     

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [15]:

# perform one-hot encoding on the data
app_df_clean = pd.get_dummies(application_df)
app_df_clean.head()
     

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALPHA PHI OMEGA,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [16]:

# Split our preprocessed data into our features and target arrays
x = app_df_clean.drop(columns=['IS_SUCCESSFUL']).values
y = app_df_clean['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y)
     

In [17]:

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [18]:
# Define the model - deep neural net
number_input_features = len(x_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               40400     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43,751
Trainable params: 43,751
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train the model
fit_model = nn.fit(x_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.4986 - accuracy: 0.7697
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4427 - accuracy: 0.7942
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4346 - accuracy: 0.7947
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4303 - accuracy: 0.7972
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4280 - accuracy: 0.7962
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4254 - accuracy: 0.7980
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4243 - accuracy: 0.7991
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4225 - accuracy: 0.8007
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4217 - accuracy: 0.7987
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4207 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.4026 - accuracy: 0.8070
Epoch 82/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4026 - accuracy: 0.8071
Epoch 83/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4022 - accuracy: 0.8075
Epoch 84/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4021 - accuracy: 0.8078
Epoch 85/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4023 - accuracy: 0.8074
Epoch 86/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4023 - accuracy: 0.8079
Epoch 87/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4025 - accuracy: 0.8070
Epoch 88/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4023 - accuracy: 0.8077
Epoch 89/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4023 - accuracy: 0.8078
Epoch 90/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4026 - accuracy: 

804/804 [==============================] - 1s 1ms/step - loss: 0.4019 - accuracy: 0.8086
Epoch 161/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4014 - accuracy: 0.8086
Epoch 162/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4012 - accuracy: 0.8088
Epoch 163/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4011 - accuracy: 0.8085
Epoch 164/200
804/804 [==============================] - 2s 2ms/step - loss: 0.4013 - accuracy: 0.8092
Epoch 165/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4015 - accuracy: 0.8090
Epoch 166/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4017 - accuracy: 0.8092
Epoch 167/200
804/804 [==============================] - 1s 2ms/step - loss: 0.4017 - accuracy: 0.8089
Epoch 168/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4013 - accuracy: 0.8091
Epoch 169/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4014 - a

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4589 - accuracy: 0.7930 - 244ms/epoch - 909us/step
Loss: 0.4589426815509796, Accuracy: 0.7930029034614563


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")   

In [26]:
# Create a random forest classifier.
rfc = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rfc = rfc.fit(x_train_scaled, y_train)

# Evaluate the model
y_predicted = rfc.predict(x_test_scaled)
print(f'Random Forest accuracy: {accuracy_score(y_test, y_predicted)*100:.2f}')

Random Forest accuracy: 77.66
